In [2]:
import numpy as np
import networkx as nx
import csv
import matplotlib.pyplot as plt
%matplotlib inline
import os

### Chargement des données

In [3]:
with open("training_set.txt", "r") as f:
    reader = csv.reader(f)
    training_set  = list(reader)
training_set = [element[0].split(" ") for element in training_set]

### Séparation train/test

In [4]:
import random
import math
training_val_idx = random.sample(range(len(training_set)),math.floor(0.9*len(training_set)))
def diff(a, b):
        b = set(b)
        return [aa for aa in a if aa not in b]
testing_val_idx=diff(range(len(training_set)),training_val_idx)

train_val = [training_set[i] for i in training_val_idx]
test_val = [training_set[i] for i in testing_val_idx]

### Récupération des IDs

In [5]:
with open("node_information.csv", "r") as f:
    reader = csv.reader(f)
    node_info  = list(reader)

IDs = [element[0] for element in node_info]
print(len(IDs))

27770


In [6]:
with open("node_information.csv", "r") as f:
    reader = csv.reader(f)
    node_info  = list(reader)

dates = [element[1] for element in node_info]
print(dates[0])

2000


### Récupération des noms d'auteurs
Retrait des parenthèses et des espaces/virgules finales

In [7]:
import re
with open("node_information.csv", "r") as f:
    reader = csv.reader(f)
    node_info  = list(reader)
IDs2 = list()
i=0
for element in node_info:
    drap = 1
    temp = re.sub('[!@#<>\\/\'1\}\{$]', '', element[3])
    temp = re.sub('\.', ' ', element[3])

    

    while drap ==1:
        idx_parenthese_ouverte = temp.find("(")
        idx_parenthese_fermee = temp.find(")")
        if idx_parenthese_ouverte != -1 and idx_parenthese_fermee != -1:
            idx_ouvert2 = temp[(idx_parenthese_ouverte+1):idx_parenthese_fermee].find("(")
            if idx_ouvert2 ==-1:
                temp = temp[0:(idx_parenthese_ouverte-1)]+temp[(idx_parenthese_fermee+1):]
            else:
                drap2=1
                idx_ouvert_temp = idx_parenthese_ouverte+1
                while drap2 ==1:
                    idx_ouvert_temp += idx_ouvert2
                    idx_ouvert2 = temp[(idx_ouvert_temp+1):idx_parenthese_fermee].find("(")
                    if idx_ouvert2==-1:
                        temp = temp[0:(idx_ouvert_temp)]+temp[(idx_parenthese_fermee+1):]
                        drap2=0
        elif idx_parenthese_ouverte != -1:
            temp = temp[0:(idx_parenthese_ouverte-1)]
        elif idx_parenthese_fermee != -1:
            temp = temp[(idx_parenthese_fermee+1):]
        elif len(temp) >1 and temp[-1]==' ':
            temp=temp[0:-1]
        elif len(temp) >1 and temp[-1]==',':
            temp=temp[0:-1]
        elif len(temp) >1 and temp[-1]=='.':
            temp=temp[0:-1]
        else :
            idx_point = temp.find(".")
            if idx_point == -1:
                drap = 0
            elif temp[idx_point+1]!=" ":
                temp=temp[0:idx_point] + ". "+ temp[(idx_point+1):]
            else:
                drap = 0

    tempsplit = temp.split(", ")
    for j in range(len(tempsplit)-1,0,-1):
        if tempsplit[j]=='':
            del(tempsplit[j])
    i+=1

    if np.mod(i,5000)==0:
        print(i)
    IDs2.append(tempsplit)

5000
10000
15000
20000
25000


### Formatage des noms
On ne garde que l'initiale et le dernier mot (qui fait office de nom)

In [8]:
res = list()
n_erreur=0
n_erreur2=0
for i in range(len(IDs2)):
    drap = 0
    temp_list = list()
    for j in range(len(IDs2[i])):
        # On récupère l'initiale et le dernier mot si c'est possible
        # Si impossible on sort des deux boucles et on ne rajoute pas le papier ?
        idx_parenthese_ouverte = IDs2[i][j].find("(")
        idx_parenthese_fermee = IDs2[i][j].find(")")
        if idx_parenthese_ouverte != -1 and idx_parenthese_fermee != -1:
            print("erreur type 1")
            IDs_ij = IDs2[i][j][0:(idx_parenthese_ouverte-1)]+IDs2[i][j][(idx_parenthese_fermee+2):]
            print(IDs2[i][j], IDs_ij)
        elif idx_parenthese_ouverte != -1:
            print("erreur type 2")
            IDs_ij = IDs2[i][j][0:(idx_parenthese_ouverte-1)]
            print(IDs2[i][j])
            print(IDs_ij)
        elif idx_parenthese_fermee != -1:
            print("erreur type 3")
            IDs_ij = IDs2[i][j][(idx_parenthese_fermee+2):]
            print(IDs2[i][j])
            print(IDs_ij)
        else : 
            IDs_ij = IDs2[i][j]
        temp = IDs_ij.split(" ")
        #print(temp)
        if temp==[""]:
            drap = 2
            break
        if temp[-1] == "" or temp[-1][0].isupper() == False:
            drap = 1
            break
        else:
            temp_list.append(temp[0][0].upper() + ". " + temp[-1])
    
    if drap == 0:
        res.append(temp_list)
    elif drap==1:
        res.append(None)
        n_erreur+=1
    else:
        res.append(None)
        n_erreur2+=1
        
print("Erreur 1 (nom bizarre) : ", n_erreur, "Erreur 2 (pas de nom) : ", n_erreur2)


Erreur 1 (nom bizarre) :  802 Erreur 2 (pas de nom) :  4033


In [6]:
len(res)

27770

## Stockage du fichier avec les noms corrigés et séparés (potentiellement inutile)

In [89]:
fo = open('node_info_rectified.csv','w')
writer = csv.writer(fo,  lineterminator = '\n')
i=0
f = open("node_information.csv", "r")
reader = csv.reader(f)

for row in reader:
    row[3] = res[i]
    writer.writerow(row)
    i+=1

fo.close()
f.close()

On selectionne les couples d'articles qui sont liés

In [9]:
couple_article = [ [id1,id2] for id1, id2, edge in train_val if int(edge) == 1 ] 
list_couple_article_without_edge = [ [id1,id2] for id1, id2, edge in train_val if int(edge) ==0 ] 

In [10]:
len(couple_article)

301426

### Création d'un dictionnaire des citations entre auteurs et des coauteurs

In [8]:
dict_id_coauthor = {}
dict_id_coauthor_strict = {}
list_coauthor = list()
for i in res:
    if i != None:
        for j in i:
            for k in i:
                    dict_id_coauthor[j]=k
                    if j != k:
                        dict_id_coauthor_strict[j]=k
                        list_coauthor.append(list([j, k]))
                    

In [137]:
G.clear()

## Création du graphe d'article

In [11]:
graph_article = nx.Graph(couple_article)
G=graph_article
#add isolated nodes
G.add_nodes_from(IDs)

### Ajout de l'attribut "auteur" à chaque noeud

In [12]:
for i in range(len(node_info)):
    if G.has_node(node_info[i][0]):
        G.node[node_info[i][0]]['author'] = res[i]

In [139]:
Graph_coauthor.clear()

## Création du graphe d'auteur (arrêtes : cocitation)

In [13]:
Graph_author = nx.Graph()
for node, data in G.nodes_iter(data=True):
    if data['author'] != None:
        for author in data['author']:
            Graph_author.add_node(author)

['M. Cvetic', 'H. Lu', 'C. Pope']

In [20]:
len(Graph_author.edge['M. Cvetic'])

1033

In [19]:
for edge in G.edges_iter():
    if G.node[edge[0]]['author'] != None and G.node[edge[1]]['author'] != None:
        for author_from in G.node[edge[0]]['author']:
            for author_to in G.node[edge[1]]['author']:
                Graph_author.add_edge(author_from,author_to)

In [78]:
len(Graph_coauthor.edge['A. Kempf'])

2

## Création du graphe de coateurs (arrêtes : corédaction)

In [21]:
Graph_coauthor = nx.Graph()
for node, data in G.nodes_iter(data=True):
    if data['author'] != None:
        for author in data['author']:
            Graph_coauthor.add_node(author)

In [22]:
for node, data in G.nodes_iter(data=True):
    if data['author'] != None:
        for author1 in data['author']:
            for author2 in data['author']:
                if author1 != author2:
                    Graph_coauthor.add_edge(author1,author2)

### Nombre de voisins en commun de l'article (normalisé)

In [23]:
frac_common_neigh=np.zeros((len(training_set),1))
has_common_neigh=np.zeros((len(training_set),1))
for i in range(len(training_set)):
    if np.mod(i,100000)==0:
        print(i)
    try:
        frac_common_neigh[i,0] = float(len(sorted(nx.common_neighbors(G,training_set[i][0],training_set[i][1]))))/float(len(sorted(nx.neighbors(G,training_set[i][0])))+len(sorted(nx.neighbors(G,training_set[i][1]))))
    except:
        frac_common_neigh[i,0]=0
    has_common_neigh[i,0]=frac_common_neigh[i,0]>0

0
100000
200000
300000
400000
500000
600000


### Nombre de citations en commun des auteurs (normalisé)

In [25]:
frac_common_neigh_author=np.zeros((len(training_set),1))
has_common_neigh_author=np.zeros((len(training_set),1))
for i in range(len(training_set)):
    if np.mod(i,100000)==0:
        print(i)
    frac_common_neigh_author[i,0]=0
    if G.node[training_set[i][0]]['author'] != None and G.node[training_set[i][1]]['author'] != None :
        sum_temp1 = 0
        sum_temp2 = 0
        for author1 in G.node[training_set[i][0]]['author']:
            for author2 in G.node[training_set[i][1]]['author']:
                try:
                    frac_common_neigh_author[i,0] += float(len(sorted(nx.common_neighbors(Graph_author,author1,author2))))
                    #/float(len(sorted(nx.neighbors(Graph_author,author1)))+len(sorted(nx.neighbors(Graph_author,author2))))
                except:
                    frac_common_neigh_author[i,0]+=0
        for author1 in G.node[training_set[i][0]]['author']:
            try:
                sum_temp1 += float(len(sorted(nx.neighbors(Graph_author,author1))))
            except:
                sum_temp1+=0
        for author2 in G.node[training_set[i][1]]['author']:
            try:
                sum_temp2 += float(len(sorted(nx.neighbors(Graph_author,author2))))
            except:
                sum_temp2+=0
        if sum_temp1 !=0 or sum_temp2 !=0:
            frac_common_neigh_author[i,0]=100*float(frac_common_neigh_author[i,0])/(sum_temp1+sum_temp2)
    has_common_neigh_author[i,0]=frac_common_neigh_author[i,0]>0

0
100000
200000
300000
400000
500000
600000


In [234]:
frac_common_neigh_coauthor

array([[ 0.        ],
       [ 4.81927711],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 0.        ],
       [ 6.06060606],
       [ 0.        ]])

### Nombre de citations en commun des coauteurs (normalisé)

In [31]:
%time
frac_common_neigh_coauthor=np.zeros((len(training_set),1))
has_common_neigh_coauthor=np.zeros((len(training_set),1))
for i in range(len(training_set)):
    if np.mod(i,100000)==0:
        print(i)
    frac_common_neigh_coauthor[i,0]=0
    if G.node[training_set[i][0]]['author'] != None and G.node[training_set[i][1]]['author'] != None :
        sum_temp1 = 0
        sum_temp2 = 0
        for author1 in G.node[training_set[i][0]]['author']:
            for author2 in G.node[training_set[i][1]]['author']:
                try:
                    frac_common_neigh_coauthor[i,0] += float(len(sorted(nx.common_neighbors(Graph_coauthor,author1,author2))))
                except:
                    frac_common_neigh_coauthor[i,0]+=0
        for author1 in G.node[training_set[i][0]]['author']:
            try:
                sum_temp1 += float(len(sorted(nx.neighbors(Graph_coauthor,author1))))
            except:
                sum_temp1+=0
        for author2 in G.node[training_set[i][1]]['author']:
            try:
                sum_temp2 += float(len(sorted(nx.neighbors(Graph_coauthor,author2))))
            except:
                sum_temp2+=0
        if sum_temp1 !=0 or sum_temp2 !=0:
            frac_common_neigh_coauthor[i,0]=100*float(frac_common_neigh_coauthor[i,0])/(sum_temp1+sum_temp2)
    has_common_neigh_author[i,0]=frac_common_neigh_author[i,0]>0

Wall time: 0 ns
0
100000
200000
300000
400000
500000
600000


In [33]:
len(train_val)

553960

In [161]:
np

<module 'numpy' from 'C:\\Anaconda\\envs\\python3\\lib\\site-packages\\numpy\\__init__.py'>

### Mots en commun dans le titre/l'abstract et nombre d'auteurs en commun

In [34]:
with open("titres_nltk.csv", "r") as f:
    reader = csv.reader(f)
    titres_nltk  = list(reader)

with open("abstracts_nltk.csv", "r") as f:
    reader = csv.reader(f)
    abstracts_nltk  = list(reader)


In [35]:
overlap_title=[]
overlap_abstract=[]
temp_diff=[]
comm_auth=[]

In [36]:
for i in range(len(training_set)):
    source = training_set[i][0]
    target = training_set[i][1]

    index_source = IDs.index(source)
    index_target = IDs.index(target)

    source_title = titres_nltk[index_source]
    target_title = titres_nltk[index_target]

    source_abstract = abstracts_nltk[index_source]
    target_abstract = abstracts_nltk[index_target]

    source_auth = res[index_source]
    target_auth = res[index_target]

    overlap_title.append(len(set(source_title).intersection(set(target_title))))
    overlap_abstract.append(len(set(source_abstract).intersection(set(target_abstract))))

    temp_diff.append(abs(int(dates[index_source]) - int(dates[index_target])))
    if source_auth!=None and target_auth != None:
        comm_auth.append(len(set(source_auth).intersection(set(target_auth))))
    else:
        comm_auth.append(0)

In [158]:
overlap_abstract

[3, 5]

In [121]:
source_info

['9510123',
 '1995',
 'an infinite number of potentials surrounding 2d black hole',
 '',
 'Phys.Lett.',
 'we found an infinite number of potentials surrounding 2d black hole according to the transmission cal t and reflection cal r coefficients for scattering of string fields off 2d black hole we can classify an infinite number of potentials into three graviton-dilaton tachyon and the other types we suggest that the discrete states from all the virasoro levels be candidates for new potentials modes']

### Nombre de citations de l'article (degré normalisée)

In [37]:
temp=G.degree()
sum_degree = sum(temp[i] for i in G.node)
degree_article={}
for i in G.node:
    degree_article[i]=10**5*temp[i]/float(sum_degree)

### Nombre de citations des auteurs/coauteurs (degré normalisé et maximum normalisé)

In [38]:
temp=Graph_author.degree()
sum_degree = sum(temp[i] for i in Graph_author.node)
degree_author={}
for i in Graph_author.node:
    degree_author[i]=10**5*temp[i]/float(sum_degree)

sum_degree_author={}
max_degree_author={}   

for i in G.nodes_iter():
    if G.node[i]['author'] != None:
        sum_degree_author[i]=sum(degree_author[author] for author in G.node[i]['author'])
        max_degree_author[i]=max(degree_author[author] for author in G.node[i]['author'])
    else:
        sum_degree_author[i]=0
        max_degree_author[i]=0

In [82]:
temp=Graph_coauthor.degree()
sum_degree = sum(temp[i] for i in Graph_coauthor.node)
degree_coauthor={}
for i in Graph_coauthor.node:
    degree_coauthor[i]=10**5*temp[i]/float(sum_degree)

sum_degree_coauthor={}
max_degree_coauthor={}   

for i in G.nodes_iter():
    if G.node[i]['author'] != None:
        sum_degree_coauthor[i]=sum(degree_coauthor[author] for author in G.node[i]['author'])
        max_degree_coauthor[i]=max(degree_coauthor[author] for author in G.node[i]['author'])
    else:
        sum_degree_coauthor[i]=0
        max_degree_coauthor[i]=0

# Base pipeline crossvalidation

In [114]:
'''
Données en commun :
Nombre de mot dans le titre
Nombre de mots dans l'abstract
Ecart temporel (abs)
Nombre d'auteurs en commun
Has common neigb (article)
Has common neigb (auteurs)
Has common neigb (coauteurs)
Nombre de voisins en commun (article)
Somme Nombre de voisins en commun (auteurs)
Max Nombre de voisins en commun (auteurs) galère
Somme Nombre de voisins en commun (coauteurs)
Max Nombre de voisins en commun (coauteurs) galère

Données perso (max, min):
Between Centralité (article)
Between Centralité(auteur)
Between Centralité(coauteur)
Degré (article)
Somme degré (auteur)
Somme degré (coauteur)
Max degré (auteur)
Max degré (coauteur)
'''

"\nDonnées en commun :\nNombre de mot dans le titre\nNombre de mots dans l'abstract\nNombre d'auteurs en commun\n\nNombre de voisins en commun (article)\nNombre de voisins en commun (auteurs)\nNombre de voisins en commun (coauteurs)\n\nDonnées perso (max, min):\nDegré (article)\nSomme degré (auteur)\nSomme degré (coauteur)\nMax degré (auteur)\nMax degré (coauteur)\n"

### Avec auteurs/coauteurs (tout le monde)

In [41]:
train_auteur = np.zeros((len(train_val),20))
test_auteur = np.zeros((len(test_val),20))

In [45]:
len(train_val)

553960

In [113]:
for i in range(len(training_val_idx)):
    idx=train_val[i]
    j=training_val_idx[i]
    train_auteur[i,0]=overlap_title[j]
    train_auteur[i,1]=overlap_abstract[j]
    train_auteur[i,2]=temp_diff[j]
    train_auteur[i,3]=comm_auth[j]
    train_auteur[i,4]=has_common_neigh[j]
    train_auteur[i,5]=has_common_neigh_author[j]
    train_auteur[i,6]=has_common_neigh_coauthor[j]
    train_auteur[i,7]=frac_common_neigh[j]
    train_auteur[i,8]=frac_common_neigh_author[j]
    train_auteur[i,9]=frac_common_neigh_coauthor[j]
    train_auteur[i,10]=max(degree_article[idx[0]],degree_article[idx[1]])
    train_auteur[i,11]=min(degree_article[idx[0]],degree_article[idx[1]])
    train_auteur[i,12]=max(sum_degree_author[idx[0]],sum_degree_author[idx[1]])
    train_auteur[i,13]=min(sum_degree_author[idx[0]],sum_degree_author[idx[1]])
    train_auteur[i,14]=max(max_degree_author[idx[0]],max_degree_author[idx[1]])
    train_auteur[i,15]=min(max_degree_author[idx[0]],max_degree_author[idx[1]])
    train_auteur[i,16]=max(sum_degree_coauthor[idx[0]],sum_degree_coauthor[idx[1]])
    train_auteur[i,17]=min(sum_degree_coauthor[idx[0]],sum_degree_coauthor[idx[1]])
    train_auteur[i,18]=max(max_degree_coauthor[idx[0]],max_degree_coauthor[idx[1]])
    train_auteur[i,19]=min(max_degree_coauthor[idx[0]],max_degree_coauthor[idx[1]])

In [114]:
for i in range(len(testing_val_idx)):
    idx=test_val[i]
    j=testing_val_idx[i]
    test_auteur[i,0]=overlap_title[j]
    test_auteur[i,1]=overlap_abstract[j]
    test_auteur[i,2]=temp_diff[j]
    test_auteur[i,3]=comm_auth[j]
    test_auteur[i,4]=has_common_neigh[j]
    test_auteur[i,5]=has_common_neigh_author[j]
    test_auteur[i,6]=has_common_neigh_coauthor[j]
    test_auteur[i,7]=frac_common_neigh[j]
    test_auteur[i,8]=frac_common_neigh_author[j]
    test_auteur[i,9]=frac_common_neigh_coauthor[j]
    test_auteur[i,10]=max(degree_article[idx[0]],degree_article[idx[1]])
    test_auteur[i,11]=min(degree_article[idx[0]],degree_article[idx[1]])
    test_auteur[i,12]=max(sum_degree_author[idx[0]],sum_degree_author[idx[1]])
    test_auteur[i,13]=min(sum_degree_author[idx[0]],sum_degree_author[idx[1]])
    test_auteur[i,14]=max(max_degree_author[idx[0]],max_degree_author[idx[1]])
    test_auteur[i,15]=min(max_degree_author[idx[0]],max_degree_author[idx[1]])
    test_auteur[i,16]=max(sum_degree_coauthor[idx[0]],sum_degree_coauthor[idx[1]])
    test_auteur[i,17]=min(sum_degree_coauthor[idx[0]],sum_degree_coauthor[idx[1]])
    test_auteur[i,18]=max(max_degree_coauthor[idx[0]],max_degree_coauthor[idx[1]])
    test_auteur[i,19]=min(max_degree_coauthor[idx[0]],max_degree_coauthor[idx[1]])

In [115]:
y_train_val = [int(training_set[i][2]) for i in training_val_idx]
y_test_val = [int(training_set[i][2]) for i in testing_val_idx]

In [116]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=5, criterion='gini', max_depth=10, min_samples_split=6, min_samples_leaf=5, max_features='auto', bootstrap=True, n_jobs=2)
clf.fit(train_auteur, y_train_val)

C:\Anaconda\envs\python3\lib\site-packages\sklearn\base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
C:\Anaconda\envs\python3\lib\site-packages\sklearn\base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
C:\Anaconda\envs\python3\lib\site-packages\sklearn\base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
C:\Anaconda\envs\python3\lib\site-packages\sklearn\base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
C:\Anaconda\envs\python3\lib\site-packages\sklearn\base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs,

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=5, min_samples_split=6,
            min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [117]:
y_pred = clf.predict(test_auteur)

In [120]:
from sklearn.metrics import precision_recall_fscore_support

In [121]:
score = precision_recall_fscore_support(y_test_val, y_pred)

In [122]:
score

(array([ 0.94121567,  0.96934112]),
 array([ 0.96362396,  0.95027296]),
 array([ 0.95228801,  0.95971234]),
 array([27848, 33704], dtype=int64))

### Avec auteurs/coauteurs (restreints)

In [92]:
train_val[0]

['2088', '9503001', '1']

In [60]:
IDs.index('2008')

219

### Sans auteurs/coauteurs

NameError: name 'training_set' is not defined

5